In [1]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split

In [2]:
input_data = np.load(open('nsmc-master/nsmc_train_input.npy', 'rb'))
label_data = np.load(open('nsmc-master/nsmc_train_label.npy', 'rb'))
prepro_configs = json.load(open('nsmc-master/data_configs.json', 'r'))

In [3]:
VOCAB_SIZE = prepro_configs['vocab_size'] + 1
EMB_SIZE = 128
BATCH_SIZE = 16
NUM_EPOCHS = 1



In [56]:
VOCAB_SIZE

43757

In [57]:
test_split = 0.1
rng_seed = 13371447

input_train, input_eval, label_train, label_eval = train_test_split(input_data, label_data, test_size=test_split, random_state=rng_seed)


In [58]:
input_train.shape, input_eval.shape, label_train.shape, label_eval.shape

((135000, 8), (15000, 8), (135000,), (15000,))

In [59]:
len(input_train), len(input_eval), len(label_train), len(label_eval)

(135000, 15000, 135000, 15000)

In [60]:
def mapping_fn(X, Y):
    input, label = {'x': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=len(input_train))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
    dataset = dataset.shuffle(buffer_size=len(input_eval))
    dataset = dataset.batch(16)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [61]:
def model_fn(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    embedding_layer = tf.keras.layers.Embedding(
                    VOCAB_SIZE,
                    EMB_SIZE)(features['x'])

    dropout_emb = tf.keras.layers.Dropout(rate = 0.2)(embedding_layer)
    
    conv = tf.layers.conv1d(
           inputs=dropout_emb,
           filters=32,
           kernel_size=3,
           padding='same',
           activation=tf.nn.relu)
  
    pool = tf.keras.layers.GlobalMaxPool1D()(conv)

    hidden = tf.keras.layers.Dense(units=250, activation=tf.nn.relu)(pool)   


    dropout_hidden = tf.keras.layers.Dropout(rate=0.2)(hidden, training = TRAIN)
    logits = tf.keras.layers.Dense(units=1)(dropout_hidden)

    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits),
            }
        )

In [62]:
est = tf.estimator.Estimator(model_fn, model_dir="data_out/checkpoint/cnn_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'data_out/checkpoint/cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x140c20048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [63]:
time_start = datetime.utcnow()
print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 

est.train(train_input_fn)

time_end = datetime.utcnow()
print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))

Experiment started at 15:47:04
.......................................
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into data_out/checkpoint/cnn_model/model.ckpt.
INFO:tensorflow:loss = 0.6927129, step = 1
INFO:tensorflow:global_step/sec: 44.9699
INFO:tensorflow:loss = 0.6622977, step = 101 (2.225 sec)
INFO:tensorflow:global_step/sec: 53.2944
INFO:tensorflow:loss = 0.5653749, step = 201 (1.876 sec)
INFO:tensorflow:global_step/sec: 52.0028
INFO:tensorflow:loss = 0.4159905, step = 301 (1.923 sec)
INFO:tensorflow:global_step/sec: 45.6218
INFO:tensorflow:loss = 0.6291093, step = 401 (2.192 sec)
INFO:tensorflow:global_step/sec: 48.2794
INFO:tensorflow:loss = 0.8714328, step = 501 (2.071 sec)
INFO:tensorflow:global_step/sec: 47.8911
INFO:tensorflow:loss = 0.42044

INFO:tensorflow:loss = 0.4923272, step = 7901 (2.316 sec)
INFO:tensorflow:global_step/sec: 41.8341
INFO:tensorflow:loss = 0.48551875, step = 8001 (2.390 sec)
INFO:tensorflow:global_step/sec: 45.0673
INFO:tensorflow:loss = 0.45027465, step = 8101 (2.219 sec)
INFO:tensorflow:global_step/sec: 41.8854
INFO:tensorflow:loss = 0.6825806, step = 8201 (2.388 sec)
INFO:tensorflow:global_step/sec: 49.5886
INFO:tensorflow:loss = 0.25300533, step = 8301 (2.016 sec)
INFO:tensorflow:global_step/sec: 50.5065
INFO:tensorflow:loss = 0.15840733, step = 8401 (1.980 sec)
INFO:tensorflow:Saving checkpoints for 8438 into data_out/checkpoint/cnn_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.27584594.
.......................................
Experiment finished at 15:50:09

Experiment elapsed time: 185.521135 seconds


In [64]:
valid = est.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-07-15:50:32
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from data_out/checkpoint/cnn_model/model.ckpt-8438
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-07-15:50:33
INFO:tensorflow:Saving dict for global step 8438: acc = 0.82446665, global_step = 8438, loss = 0.3820936
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8438: data_out/checkpoint/cnn_model/model.ckpt-8438


In [66]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'

test_input_data = np.load(open('nsmc-master/' + INPUT_TEST_DATA, 'rb'))
test_label_data = np.load(open('nsmc-master/' + LABEL_TEST_DATA, 'rb'))

In [67]:
def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((test_input_data, test_label_data))
    dataset = dataset.batch(16)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [68]:
predict = est.evaluate(test_input_fn) 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-07-15:51:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from data_out/checkpoint/cnn_model/model.ckpt-8438
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-07-15:51:23
INFO:tensorflow:Saving dict for global step 8438: acc = 0.8261, global_step = 8438, loss = 0.3843277
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8438: data_out/checkpoint/cnn_model/model.ckpt-8438
